## Writing GRIB to an FDB target

This example demonstrates how to **write earthkit-data GRIB fields into an FDB**. 

FDB (Fields DataBase) is a domain-specific object store developed at ECMWF for storing, indexing and retrieving GRIB data. For more information on FBD please consult the following pages:

- [FDB](https://fields-database.readthedocs.io/en/latest/)
- [pyfdb](https://pyfdb.readthedocs.io/en/latest/)

#### Setting up the target FDB

In this example we will create an FDB in the current folder using the schema taken from the pyfdb test suite. To do so first we need to ensure the directory exists. Next, we have to specify the configuration.

In [1]:
import os
fdb_schema = "./default_fdb_schema"
fdb_dir = "./_fdb_target_demo"
os.makedirs(fdb_dir, exist_ok=True)

config = {"type":"local",
     "engine":"toc",
     "schema":fdb_schema,
     "spaces":[{"handler":"Default",
                "roots":[{"path":fdb_dir}]}]}

Working with FDB requires pyfdb and fdb to be installed. The path to the fdb installation should also be set e.g. via the FDB5_DIR environment variable.

#### Getting the input data

In [2]:
import earthkit.data as ekd
ds = ekd.from_source("sample", "tuv_pl.grib")
ds.describe()

tuv_pl.grib:   0%|          | 0.00/4.22k [00:00<?, ?B/s]

,,level,date,time,step,paramId,class,stream,type,experimentVersionNumber
shortName,typeOfLevel,,,,,,,,,
t,isobaricInhPa,"1000,300,...",20180801,1200,0,130,od,oper,an,0001
u,isobaricInhPa,"1000,300,...",20180801,1200,0,131,od,oper,an,0001
v,isobaricInhPa,"1000,300,...",20180801,1200,0,132,od,oper,an,0001


#### Using to_target() on the data object

In [3]:
# writing a field
ds[0].to_target("fdb", config=config)

# writing a whole fieldlist
ds.to_target("fdb", config=config)

In [4]:
# checking the result
request = {
    'class': 'od',
    'expver': '0001',
    'stream': 'oper',
    'date': '20180801',
    'time': 1200,
    'domain': 'g',
    'type': 'an',
    'levtype': 'pl',
    'levelist': 500,
    'step': 0,
    'param': [131,132]
}

ekd.from_source("fdb", request, config=config, stream=False).ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,u,isobaricInhPa,500,20180801,1200,0,an,0,regular_ll
1,ecmf,v,isobaricInhPa,500,20180801,1200,0,an,0,regular_ll


In [5]:
# setting GRIB keys for the output
ds.to_target("fdb", config=config, metadata={"date": 20250108})

In [6]:
# checking the result
request_1 = dict(**request)
request_1.update({"date": 20250108})
ekd.from_source("fdb", request_1, config=config, stream=False).ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,u,isobaricInhPa,500,20250108,1200,0,an,0,regular_ll
1,ecmf,v,isobaricInhPa,500,20250108,1200,0,an,0,regular_ll


#### Using a Target object

In [ ]:
# basic usage
target = ekd.create_target("fdb", config=config)
target.write(ds)
target.flush()

# can be used as a context manager, no need to call flush() in the end
with ekd.create_target("fdb", config=config) as target:
    target.write(ds)

# a fieldlist can be written field by field into the target
with ekd.create_target("fdb", config=config) as target:
    for f in ds:
        target.write(f)